<a href="https://colab.research.google.com/github/malonsos/ClimateBert/blob/main/ClimateBert_CEOspolide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
 %pip install transformers

In [2]:
import os
os.getcwd()

'/content'

In [3]:
%mkdir Climate_bert

In [4]:
from google.colab import files

uploaded = files.upload()

Saving 2010-A.txt to 2010-A.txt
Saving 2010-AAL.txt to 2010-AAL.txt
Saving 2010-AAP.txt to 2010-AAP.txt


In [15]:
import re
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

# List to store results
results = []

# Iterate through uploaded files
for filename, content in uploaded.items():
    # Decode content to UTF-8
    text = content.decode('utf-8')
    # Split text into paragraphs at each period
    paragraphs = re.split('\n\n', text)

    # Load pre-trained model and tokenizer
    model_name = "climatebert/distilroberta-base-climate-detector"
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

    # Create text classification pipeline
    pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=-1)

    # Apply the pipeline to each paragraph
    for i, paragraph in enumerate(paragraphs):
        # Tokenize the paragraph and handle long sequences
        tokens = tokenizer(paragraph, max_length=512, truncation=True, return_overflowing_tokens=True)

        # Iterate over tokenized sequences
        for j in range(len(tokens["input_ids"])):
            input_ids = tokens["input_ids"][j]
            attention_mask = tokens["attention_mask"][j]

            # Convert to string
            truncated_text = tokenizer.decode(input_ids, skip_special_tokens=True)

            # Apply the pipeline to the truncated text
            output = pipe(truncated_text)[0]

            # Append results to the list
            results.append({
                'File Name': filename,
                'Part Number': f"{i + 1}.{j + 1}",
                'Text': truncated_text,
                'Label': output['label'],
                'Score': output['score']
            })

# Create a DataFrame from the results
df = pd.DataFrame(results)

# Add column counting words
df['WordCount'] = df['Text'].apply(lambda x: len(str(x).split()))

# Save the DataFrame to a CSV file
csv_file_path = '/content/Climate_bert/classification_results.csv'
df.to_csv(csv_file_path, index=False)

print(f"Results saved to: {csv_file_path}")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Results saved to: /content/Climate_bert/classification_results.csv


In [16]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.2 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.6 MB/s eta 0:00:00


In [17]:
#specificity
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from transformers.pipelines.pt_utils import KeyDataset
import datasets
from tqdm.auto import tqdm
from google.colab import files

# Upload your Excel file to the Colab environment
uploaded = files.upload()

# Load the Excel file into a Pandas DataFrame
df = pd.read_excel("classification_results_ejemplo_mas.xlsx")

# Filter rows where label is "yes"
df_yes = df[df['Label'] == 'yes']

# Extract the text from the filtered DataFrame
texts = df_yes['Text'].tolist()

# Initialize model and tokenizer
model_name = "climatebert/distilroberta-base-climate-specificity"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

# Initialize pipeline
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=-1)

# Process the texts through the pipeline
for out in tqdm(pipe(texts, padding=True, truncation=True)):
    print(out)

Saving classification_results_ejemplo_mas.xlsx to classification_results_ejemplo_mas.xlsx


config.json:   0%|          | 0.00/895 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/329M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/4.48k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


  0%|          | 0/28 [00:00<?, ?it/s]

{'label': 'non', 'score': 0.8979853391647339}
{'label': 'spec', 'score': 0.5335308313369751}
{'label': 'spec', 'score': 0.5163736343383789}
{'label': 'non', 'score': 0.6532329320907593}
{'label': 'spec', 'score': 0.500647246837616}
{'label': 'non', 'score': 0.775956928730011}
{'label': 'non', 'score': 0.7200310230255127}
{'label': 'non', 'score': 0.900140106678009}
{'label': 'non', 'score': 0.7997185587882996}
{'label': 'non', 'score': 0.7779151797294617}
{'label': 'non', 'score': 0.9256983399391174}
{'label': 'non', 'score': 0.8004698753356934}
{'label': 'non', 'score': 0.9481915235519409}
{'label': 'non', 'score': 0.7314825654029846}
{'label': 'spec', 'score': 0.58018559217453}
{'label': 'non', 'score': 0.8140643239021301}
{'label': 'non', 'score': 0.9482415914535522}
{'label': 'non', 'score': 0.9695038795471191}
{'label': 'non', 'score': 0.9655869603157043}
{'label': 'non', 'score': 0.5937685370445251}
{'label': 'non', 'score': 0.8382936120033264}
{'label': 'non', 'score': 0.9577466

In [18]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm
from google.colab import files

# Upload your Excel file to the Colab environment
uploaded = files.upload()

# Load the Excel file into a Pandas DataFrame
df = pd.read_excel("classification_results_ejemplo_mas.xlsx")

# Filter rows where label is "yes"
df_yes = df[df['Label'] == 'yes']

# Extract the text from the filtered DataFrame
texts = df_yes['Text'].tolist()

# Initialize model and tokenizer
model_name = "climatebert/distilroberta-base-climate-specificity"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, max_len=512)

# Initialize pipeline
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=-1)

# Process the texts through the pipeline
results = []
for i, out in tqdm(enumerate(pipe(texts, padding=True, truncation=True))):
    results.append({
        'File Name': df_yes.iloc[i]['File Name'],
        'Part Number': df_yes.iloc[i]['Part Number'],
        'Text': df_yes.iloc[i]['Text'],
        'Label': out['label'],
        'Score': out['score']
    })

# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Add column counting words
results_df['WordClimate'] = results_df['Text'].apply(lambda x: len(str(x).split()))

# Save results to CSV file
results_df.to_csv("classification_results1.csv", index=False)

# Download the CSV file
files.download("classification_results1.csv")

Saving classification_results_ejemplo_mas.xlsx to classification_results_ejemplo_mas (1).xlsx


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


0it [00:00, ?it/s]

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>